In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import category_encoders as ce
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb
import lightgbm

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError


## Load data

In [ ]:
df_2022_sodas = pd.read_csv('C:/Users/20193222/OneDrive - TU Eindhoven/DIiA Data/data_2022_sodas.csv')
df_2023_sodas = pd.read_csv('C:/Users/20193222/OneDrive - TU Eindhoven/DIiA Data/data_2023_sodas.csv')

## Preprocessing

In [ ]:
df_all = pd.concat([df_2022_sodas, df_2023_sodas], ignore_index=True)

#drop Unnamed: 0 and Clouds columns
df_all = df_all.drop(['Unnamed: 0', 'Clouds'], axis=1)

In [ ]:
#get top 10 most popular sodas
sodas = df_all['SaleDescription'].value_counts()[:10].index.tolist()

In [ ]:
# adding day number (1 to 30 per month) and week number (1 to 52 weeks per year)
df_all['Day'] = pd.to_datetime(df_all['Day'],errors ='coerce')
df_all['Week_num'] = df_all['Day'].dt.isocalendar().week
df_all['Day_num'] = df_all['Day'].dt.day

In [ ]:
# replace season names with numbers
df_all['Season'] = df_all['Season'].replace(['Winter', 'Spring', 'Summer', 'Autumn'], [0, 1, 2, 3])

In [ ]:
def replace_null(df):
    ''' Replace null values with 'Not_holiday' and 'Not_school_holiday' '''
    df['Holiday'].fillna('Not_holiday', inplace = True)
    df['SchoolHoliday'].fillna('Not_school_holiday', inplace = True)
    return df

In [ ]:
def preprocess_product(df, product):
    ''' Preprocess data for a specific product in order to train a model for that product '''
    df1 = df[df['SaleDescription'] == product]
    dummydb = df1.groupby('Day', as_index=False).last()
    
    num_sales = df1.groupby(['Day'], as_index=False)['SaleDescription'].count()
    dummydb['Num_sale'] = num_sales['SaleDescription']
    
    df_model = dummydb[['Weekday','Day_num', 'Week_num', 'Month', 'Year', 'Weekend', 'Season', 'Holiday', 'SchoolHoliday', 'HolidayBool', 'Avg Wind Speed', 
                        'Avg Temp', 'Min Temp', 'Max Temp', 'Sun Hours', 'Hours of rain', 'Rain','RainBool', 'Num_sale','Day']]
    
    
    return df_model

In [ ]:
def standardize(df, column):
    '''Standardize a column in a dataframe'''
    df_std = df.copy()
    df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
    return df_std

## Model Selection

### Train Test Split

In [ ]:
def train_test_set(df, date):
    ''' Split data into train and test set based on a given date '''
    train_df = df[df['Day'] < date]
    test_df = df[df['Day'] > date]

    X_train = train_df.drop(['Num_sale','Day'], axis = 1)
    y_train = train_df[['Num_sale']]
    X_test = test_df.drop(['Num_sale','Day'], axis = 1)
    y_test = test_df[['Num_sale']]
    
    
    return X_train, y_train, X_test, y_test

def train_test_df_for_XGBoost_lightGBM(df, date):
    ''' Preprocess data for XGBoost and LightGBM models and split into train and test set based on a given date '''
    # replacing all null values
    replace_null(df)
    
    # Converting day_num, Month, year, weekday,Weekend, holidayBool, rainBool as categorical column
    df['Month'] = df['Month'].map(str)
    df['Year'] = df['Year'].map(str)
    df['Weekday'] = df['Weekday'].map(str)
    df['HolidayBool'] = df['HolidayBool'].map(str)
    df['RainBool'] = df['RainBool'].map(str)
    df['Day_num'] = df['Day_num'].map(str)
    df['Weekend'] = df['Weekend'].map(str)
    df['Week_num'] = df['Week_num'].map(str)
    
    train_df = df[df['Day'] < date]
    test_df = df[df['Day'] > date]
    
    X_train = train_df.drop(['Num_sale','Day'], axis = 1)
    y_train = train_df[['Num_sale']]
    X_test = test_df.drop(['Num_sale','Day'], axis = 1)
    y_test = test_df[['Num_sale']]
    
    # Extract text features
    cats = list(X_train.loc[:,X_train.dtypes == 'object'].columns.values)

    # Convert to Pandas category
    for col in cats:
        X_train[col] = X_train[col].astype('category')
        X_test[col] = X_test[col].astype('category')
        
    return X_train,y_train,X_test,y_test

def train_test_df_for_NN(df, date):
    ''' Preprocess data for Neural Network model and split into train and test set based on a given date '''
    # replacing all null values
    replace_null(df)
    
    # Converting day_num, Month, year, weekday,Weekend, holidayBool, rainBool as categorical column
    df['Month'] = df['Month'].map(str)
    df['Year'] = df['Year'].map(str)
    df['Weekday'] = df['Weekday'].map(str)
    df['HolidayBool'] = df['HolidayBool'].map(str)
    df['RainBool'] = df['RainBool'].map(str)
    df['Day_num'] = df['Day_num'].map(str)
    df['Weekend'] = df['Weekend'].map(str)
    df['Week_num'] = df['Week_num'].map(str)
    
    train_df = df[df['Day'] < date]
    test_df = df[df['Day'] > date]
    
    train_df_1 = train_df.drop(['Day'],axis = 1)
    test_df_1 = test_df.drop(['Day'],axis = 1)
    col_list = train_df_1.select_dtypes(include=['object']).columns.to_list()

    # Encoding categorical columns
    encoder =  ce.BinaryEncoder(cols = col_list, return_df = True )
    train_df_encoded =  encoder.fit_transform(train_df_1)
    test_df_encoded = encoder.transform(test_df_1)
    
    X_train = train_df_encoded.drop(['Num_sale'], axis = 1)
    y_train = train_df_encoded[['Num_sale']]
    X_test = test_df_encoded.drop(['Num_sale'], axis = 1)
    y_test = test_df_encoded[['Num_sale']]
    
    return X_train,y_train,X_test,y_test

### Model Evaluation and Feature Importance

In [ ]:
def evaluate_model(model, X_test, y_test, print_metrics=False):
    ''' Evaluate model based on MAE, MSE, RMSE and R2 score '''
    y_pred = model.predict(X_test)
    
    metrics = {'MAE': mean_absolute_error(y_test, y_pred),
               'MSE': mean_squared_error(y_test, y_pred),
               'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
               'R2': r2_score(y_test, y_pred)}
    
    if print_metrics:
        print('Mean Absolute Error:', metrics['MAE'])  
        print('Mean Squared Error:', metrics['MSE'])  
        print('Root Mean Squared Error:', metrics['RMSE'])
        print('R2 score:', metrics['R2'])
        
    return metrics
    
def plot_feature_importance(model, X_train):
    ''' Plot feature importance of a model '''
    feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
    feature_importances.iloc[::-1].plot(kind='barh', figsize=(10, 6))
    plt.show()
    
def plot_prediction(model, X_test, y_test):
    ''' Plot true and predicted values '''
    y_pred = model.predict(X_test)
    plt.plot(y_test, label='True')
    plt.plot(y_pred, label='Prediction')
    plt.legend()
    plt.show()

### Neural Network

In [ ]:
def Neural_Network(X_train,y_train,X_test,y_test):
    ''' Train a Neural Network model '''
    hidden_units1 = 10
    hidden_units2 = 10
    hidden_units3 = 10
    learning_rate = 0.01
    # Creating model using the Sequential in tensorflow
    def build_model_using_sequential():
        model = Sequential([
        Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
        Dense(1, kernel_initializer='normal', activation='linear')
          ])
        return model
    
    # build the model
    model_NN = build_model_using_sequential()
    # loss function
    msle = MeanSquaredLogarithmicError()
    model_NN.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
    )
    # train the model
    history = model_NN.fit(
        X_train.values, 
        y_train.values, 
        epochs=150, 
        batch_size=16,
        validation_split=0.2
    )
    
    return model_NN

### Find best combination of features for a given model
For each model, multiple combinations of features are tested, and the best combination is returned.

In [ ]:
def model_selection(X_train, X_test, y_train, y_test, model):
    ''' Find best feature combination for a given model '''
    all_features = ['Weekday','Day_num', 'Week_num', 'Month', 'HolidayBool', 'Avg Wind Speed', 
                    'Avg Temp', 'Min Temp', 'Max Temp', 'Sun Hours', 'Hours of rain', 'Rain']

    # These features have been defined as optional features
    removable_features = ['Year', 'Weekend', 'Season', 'RainBool']
    
    results = {}
    models = {}

    # train model for all possible feature combinations
    for r in range(len(removable_features) + 1):
        varying_combinations = combinations(removable_features, r)
        for combo in varying_combinations:
            features_combination = all_features.copy()
            for item in list(combo):
                features_combination.append(item)
            X_train_temp = X_train[features_combination]
            X_test_temp = X_test[features_combination]
            
            # XGBoost requires a different data format
            if type(model) == type(xgb.XGBRegressor()):
                model.fit(X_train_temp, y_train, eval_set=[(X_test, y_test)])
                models[combo] = model
            else:
                model.fit(X_train_temp, y_train)
                models[combo] = model
                
            # evaluate model
            metrics = evaluate_model(model, X_test_temp, y_test, print_metrics=False)
            results[combo] = metrics

    #find best feature combination and its metrics based on R2 score
    best_feature_combo = max(results, key=lambda k: results[k]['R2'])
    best_features = all_features.copy()
    for item in list(best_feature_combo):
        best_features.append(item)
    best_feature_score = results[best_feature_combo]
    best_model = models[best_feature_combo]
    
    return best_features, best_feature_score, best_model


### Find best model per soda
For each soda, all models are tested and the best one is returned.

In [ ]:
def run_models(df_model, date):
    ''' Run all models for a given product '''
    
    # run linear regression, random forest, support vector regression, and decision tree
    X_train, y_train, X_test, y_test = train_test_set(df_model, date)
    
    # standardize y_train and y_test - this is used for some of the models
    std_y_train = standardize(y_train, 'Num_sale')
    std_y_test = standardize(y_test, 'Num_sale')
    
    lr = LinearRegression()
    lr_features, lr_results, lr_model = model_selection(X_train, X_test, y_train, y_test, lr)
    
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rf_features, rf_results, rf_model = model_selection(X_train, X_test, std_y_train, std_y_test, rf)
    
    svr = SVR(kernel='rbf', degree = 3, C=1000, gamma=0.01, epsilon=0.1)
    svr_features, svr_results, svr_model = model_selection(X_train, X_test, std_y_train, std_y_test, svr)
    
    dt = DecisionTreeRegressor(random_state = 42)
    dt_features, dt_results, dt_model = model_selection(X_train, X_test, y_train, y_test, dt)
    
    # run XGBoost and LightGBM
    X_train2, y_train2, X_test2, y_test2 = train_test_df_for_XGBoost_lightGBM(df_model, date)
    
    #TODO: fix xgboost --> this model works in the 'combined_model.ipynb' file
    # xgb_model = xgb.XGBRegressor(enable_categorical=True, tree_method= 'hist', gamma = 1, random_state = 101, learning_rate = 0.001, max_depth = 30, early_stopping_rounds = 10)
    # xgb_features, xgb_results, xgb_model = model_selection(X_train2, X_test2, y_train2, y_test2, xgb_model)
    
    lgbm = lightgbm.LGBMRegressor()
    lgbm_features, lgbm_results, lgbm_model = model_selection(X_train2, X_test2, y_train2, y_test2, lgbm)
    
    # run neural network
    X_train3, y_train3, X_test3, y_test3 = train_test_df_for_NN(df_model, date)
    
    #TODO: fix neural network --> this model works in the 'combined_model.ipynb' file
    # NN_model = Neural_Network(X_train3,y_train3,X_test3,y_test3)
    # nn_features, nn_results, nn_model = model_selection(X_train3, X_test3, y_train3, y_test3, NN_model)
    
    # save results
    metrics = {'Linear Regression': lr_results,
                'Random Forest': rf_results,
                'Support Vector Regression': svr_results,
                'Decision Tree': dt_results,
                # 'XGBoost': xgb_results,
                'LightGBM': lgbm_results,
                # 'Neural Network': nn_results,
                }
    
    features = {'Linear Regression': lr_features,
                'Random Forest': rf_features,
                'Support Vector Regression': svr_features,
                'Decision Tree': dt_features,
                # 'XGBoost': xgb_features,
                'LightGBM': lgbm_features,
                # 'Neural Network': nn_features,
                }
    
    models = {'Linear Regression': lr_model,
                'Random Forest': rf_model,
                'Support Vector Regression': svr_model,
                'Decision Tree': dt_model,
                # 'XGBoost': xgb_model,
                'LightGBM': lgbm_model,
                # 'Neural Network': nn_model,
                }
    
    # find best model
    best_model = max(metrics, key=lambda k: metrics[k]['R2'])
    best_model_score = metrics[best_model]
    best_model_features = features[best_model]
    
    return metrics, features, models, best_model, best_model_score, best_model_features

## Run pipeline
This pipeline runs for each product (soda) in the data, and includes preprocessing, running the models and feature selection steps. It also makes predictions using the best model for each soda and saves them to a xlsx file per soda.

In [ ]:
metrics_dict = {}

for soda in sodas:
    df_predictions = pd.DataFrame(columns=['Day', 'Num_sale', 'Predicted_sales'])
    df_model = preprocess_product(df_all, soda)
    metrics, features, models, best_model, best_model_score, best_model_features = run_models(df_model, '2023-05-30')
    
    print(f'Best model for {soda} is: {best_model}')
    metrics_dict[soda] = metrics
    
    #get predictions with best_model
    model = models[best_model]
    days = list(df_model[df_model['Day'] > '2023-05-30']['Day'])
    
    if best_model == 'LightGBM':
        X_train, y_train, X_test, y_test = train_test_df_for_XGBoost_lightGBM(df_model, '2023-05-30')
    elif best_model == 'Neural Network':
        X_train, y_train, X_test, y_test = train_test_df_for_NN(df_model, '2023-05-30')
    else: 
        X_train, y_train, X_test, y_test = train_test_set(df_model, '2023-05-30')
        
    pred_data = X_test
    pred_features = pred_data[['Weekday','Day_num', 'Week_num', 'Month', 'HolidayBool', 'Avg Wind Speed', 
            'Avg Temp', 'Min Temp', 'Max Temp', 'Sun Hours', 'Hours of rain', 'Rain', 
            'Year', 'Weekend', 'Season', 'RainBool']]
    pred = model.predict(pred_features)
    
    
    df_predictions['Num_sale'] = y_test
    df_predictions['Day'] = days
    df_predictions['predict'] = [int(x) for x in pred]
    df_predictions.reset_index(drop=True, inplace=True)
    
    # dynamically allocating file name
    filename = f'{soda}.xlsx'

    df_predictions.to_excel(filename, index = False)

## Model Evaluation

### Only R2 scores

In [ ]:
#This is the model performance for the models tested in the combined_model.ipynb file
model_performance = pd.read_csv('model_performance.csv')
model_performance_small = model_performance.copy()
model_performance_small['R2'] = model_performance_small['R2_score'].round(2)
model_performance_small = model_performance_small.drop(columns=['MSE','RMSE','MAE', 'R2_score'])

In [ ]:
#get highest R2 score for each soda
model_performance_small = model_performance_small.sort_values(by=['R2'], ascending=False)
model_performance_small = model_performance_small.drop_duplicates(subset=['Product_Name'], keep='first')
model_performance_small = model_performance_small.reset_index(drop=True)

In [ ]:
#find best model for each soda
r2_scores = {}
for soda in sodas:
    r2_scores[soda] = {}
    for model in metrics_dict[soda]:
        if 'R2' in metrics_dict[soda][model]:
            r2_scores[soda][model] = metrics_dict[soda][model]['R2']

r2_scores_df = pd.DataFrame(r2_scores)
#pivot table
r2_scores_df = r2_scores_df.transpose()
r2_scores_df['Best model'] = r2_scores_df.idxmax(axis=1)

r2_scores_df

In [ ]:
r2_scores = r2_scores_df.reset_index()
r2_scores = r2_scores.rename(columns={'index': 'Product_Name'})

#get max r2 score for each soda
r2_scores['R2_score'] = r2_scores[['Linear Regression', 'Decision Tree']].max(axis=1).round(2)
r2_scores = r2_scores.drop(columns=['Linear Regression', 'Decision Tree'])

#merge with model_performance
merged_scores = pd.merge(model_performance_small, r2_scores, on='Product_Name')
merged_scores

df_r2_scores = pd.DataFrame(columns=['Product_Name', 'Model', 'R2_score'])
for i in range(len(merged_scores)):
    if merged_scores['R2'][i] > merged_scores['R2_score'][i]:
        df_r2_scores.loc[i] = [merged_scores['Product_Name'][i], merged_scores['Model_name'][i], merged_scores['R2'][i]]
    else:
        df_r2_scores.loc[i] = [merged_scores['Product_Name'][i], merged_scores['Best model'][i], merged_scores['R2_score'][i]]
        
df_r2_scores

### All metrics

In [ ]:
metrics_dict_df = pd.DataFrame(metrics_dict)
#pivot table
metrics_dict_df = metrics_dict_df.transpose()

In [ ]:
#create dataframe per soda with all model metrics
df_metrics = pd.DataFrame(columns=['Product_Name', 'Model_name', 'MAE', 'MSE', 'RMSE', 'R2_score'])
for soda in sodas:
    df_metrics = pd.concat([df_metrics, model_performance[model_performance['Product_Name'] == soda]], ignore_index=True)
    #also add models from metrics_dict
    for i in range(0, len(metrics_dict_df)):
        if metrics_dict_df.index[i] == soda:
            for model in metrics_dict_df.columns:
                df_model = pd.DataFrame(metrics_dict_df.loc[soda, model], index=[0])
                df_model['Product_Name'] = soda
                df_model['Model_name'] = model
                df_model.rename(columns={'MAE': 'MAE', 'MSE': 'MSE', 'RMSE': 'RMSE', 'R2': 'R2_score'}, inplace=True)
                df_metrics = pd.concat([df_metrics, df_model], ignore_index=True)
#round everything to 2 decimals
df_metrics = df_metrics.round(2)

#split dataframe into dataframe per product_name
for soda in sodas:
    df_metrics[df_metrics['Product_Name'] == soda].to_csv(f'{soda}_metrics.csv', index=False)

## Feature importance analysis

In [ ]:
soda = 'FANTA EXOTIC 330ML' #change to soda of your choice
metrics_dict = {}
df_predictions = pd.DataFrame(columns=['Day', 'SaleDescription', 'Num_sale', 'predict'])

df_model = preprocess_product(df_all, soda)
X_train_imp, y_train_imp, X_test_imp, y_test_imp = train_test_df_for_XGBoost_lightGBM(df_model, '2023-05-30') #change to data and train/test split of your choice

lgbm = lightgbm.LGBMRegressor() #change to model of your choice
lgbm.fit(X_train_imp, y_train_imp, eval_set=[(X_test_imp, y_test_imp)])
plot_feature_importance(lgbm, X_train_imp)